In [1]:
import numpy as np
import time
import random
import queue
from operator import itemgetter

numberOfMachinesInStage = [3, 1]
procTimes = [
    [30, 20],
    [30, 20],
    [30, 20],
    [40, 15],
    [40, 15],
    [50, 30],
    [40, 15],
    [35, 20],
    [35, 20],
    [35, 30],
    [40, 5],
    [40, 5],
    [40, 30],
    [40, 20],
    [40, 20],
    [40, 20],
    [40, 15],
    [35, 30],
    [40, 20],
    [40, 20]
]


def createRandomPop(pop):
    randomPop = []
    for i in range(pop):
        p = list(np.random.permutation(len(procTimes)))
        while p in randomPop:
            p = list(np.random.permutation(len(procTimes)))
        randomPop.append(p)

    return randomPop


def initialization(pop):
    return createRandomPop(pop)


def findFirstNotBusyMachine(machines):
    for index, machine in enumerate(machines):
        if not machine:
            return index

    return -1


def getAvailableMachines(machines):
    available = []
    for i in range(len(machines)):
        if machines[i]:
            available.append(i)

    return available


def getEndTimeOfJob(machineTracker, job):
    for machineItem in machineTracker:
        if machineItem[4] == job:
            return machineItem[1]


def calculateCurrentEndTime(machineTracker, job, i):
    items = sorted(machineTracker, key=itemgetter(1))
    endTime = items[i][1]
    oldEndTime = getEndTimeOfJob(machineTracker, job)
    if oldEndTime < endTime:
        return endTime
    else:
        return oldEndTime


def getMinimumEndTimeFromSecondStage(machineTracker):
    endTime = machineTracker[0][1]
    for i in range(len(machineTracker)):
        if machineTracker[i][1] <= endTime:
            endTime = machineTracker[i][1]

    return endTime


def getMinimumEndTimeFromFirstStage(machineTracker):
    endTime = machineTracker[0][1]
    for i in range(len(machineTracker)):
        if machineTracker[i][1] <= endTime:
            endTime = machineTracker[i][1]

    return endTime


def findProcessedJobsInFirstStage(machineTracker, currentTime):
    jobItem = []
    for i in range(len(machineTracker)):
        if machineTracker[i][2] == 0 and currentTime >= machineTracker[i][1]:
            jobItem.append(machineTracker[i])

    return jobItem


def findProcessedJobsInSecondStage(machineTracker, currentTime):
    jobItem = []
    for i in range(len(machineTracker)):
        if machineTracker[i][2] == 1 and currentTime >= machineTracker[i][1]:
            jobItem.append(machineTracker[i])

    return jobItem


def removeFromJobs(jobs, job):
    newJFirst = jobs
    index = 0
    for i in range(len(newJFirst)):
        if newJFirst[i][4] == job:
            index = i

    newJFirst.pop(index)
    return newJFirst


def initiateCmax(sol):
    currentTime = 9999999
    availableMachines = []
    for i in numberOfMachinesInStage:
        item = []
        for j in range(i):
            item.append(True)

        availableMachines.append(item)
    processedJobs = []
    machineTracker = []  # start, end, stage, machine, job
    qFirst = queue.Queue()
    qSecond = queue.Queue()
    jFirst = []
    jSecond = []
    for i in range(len(sol)):
        qFirst.put(sol[i])
    for i in range(len(availableMachines[0])):
        job = qFirst.get()
        start = 0
        end = 0 + procTimes[job][0]
        stage = 0
        machine = i
        obj = [start, end, stage, machine, job]
        jFirst.append(obj)
        machineTracker.append(obj)
        if end <= currentTime:
            currentTime = end
        availableMachines[0][i] = False
    return availableMachines, currentTime, jFirst, jSecond, machineTracker, processedJobs, qFirst, qSecond


def getAndProcessFinishedJobs(availableMachines, currentTime, jSecond, processedJobs, jFirst, qFirst):
    finishedJobsInSecondStage = findProcessedJobsInSecondStage(jSecond, currentTime)
    # if all machines are being used and there is no finished jobs in second stage
    # then set current time to minimum end time from second stage
    # and get new finished jobs in second stage with new current time
    avMachineInSecondStage = getAvailableMachines(availableMachines[1])
    if (len(finishedJobsInSecondStage) == 0 and len(avMachineInSecondStage) == 0) or (
            len(jFirst) == 0 and qFirst.empty()) and len(jSecond) != 0:
        currentTime = getMinimumEndTimeFromSecondStage(jSecond)
        finishedJobsInSecondStage = findProcessedJobsInSecondStage(jSecond, currentTime)
    # if jobs in second stage processed, then send them to processed machines,
    # and make these machines available.
    for k in range(len(finishedJobsInSecondStage)):
        finishedJob = finishedJobsInSecondStage[k][4]
        jSecond = removeFromJobs(jSecond, finishedJob)
        finishedMachine = finishedJobsInSecondStage[k][3]
        availableMachines[1][finishedMachine] = True
        processedJobs.append(0)
    return currentTime, jSecond


def sendJobsToFirstStage(availableMachines, currentTime, jFirst, machineTracker, qFirst):
    avMacInFirstStage = getAvailableMachines(availableMachines[0])
    for i in range(len(avMacInFirstStage)):
        if not qFirst.empty():
            job = qFirst.get()
            stage = 0
            machine = avMacInFirstStage[i]
            start = currentTime
            end = currentTime + procTimes[job][0]
            obj = [start, end, stage, machine, job]
            machineTracker.append(obj)
            jFirst.append(obj)
            # 40
            availableMachines[0][machine] = False


def sendJobToSecondStage(avMacInSecondStage, availableMachines, currentTime, i, jSecond, job, machineTracker):
    stage = 1
    machine = avMacInSecondStage[i]
    start = currentTime
    end = currentTime + procTimes[job][1]
    # put smartly calculated obj into machine tracker
    obj = [start, end, stage, machine, job]
    jSecond.append(obj)
    machineTracker.append(obj)
    # make that machine not available
    availableMachines[1][machine] = False


def sendJobsToQ(availableMachines, jFirst, jSecond, qSecond):
    endTime = getMinimumEndTimeFromSecondStage(jSecond)
    jobsWithIndices = findProcessedJobsInFirstStage(jFirst, endTime)
    for i in range(len(jobsWithIndices)):
        job = jobsWithIndices[i][4]
        jobIndex = jobsWithIndices[i][3]
        qSecond.put(job)
        # now, corresponding machine in first stage is available
        jFirst = removeFromJobs(jFirst, job)
        availableMachines[0][jobIndex] = True
    return jFirst


def calculateObj(sol):
    availableMachines, currentTime, jFirst, jSecond, machineTracker, processedJobs, qFirst, qSecond = initiateCmax(sol)

    while len(processedJobs) != len(sol):
        # dealing with second stageprocessedJobs
        avMacInSecondStage = getAvailableMachines(availableMachines[1])
        # if there is no available machine then put finished jobs in stage one into qSecond (smartly)
        if len(avMacInSecondStage) == 0:
            jFirst = sendJobsToQ(availableMachines, jFirst, jSecond, qSecond)
        else:
            # loop through number of available machines in second stage
            jobsWithIndices = findProcessedJobsInFirstStage(jFirst, currentTime)
            for i in range(min(len(avMacInSecondStage), len(jobsWithIndices))):
                if not qSecond.empty():
                    job = qSecond.get()
                    currentTime = calculateCurrentEndTime(machineTracker, job, i)
                else:
                    # get least end time job
                    # find index of used job in first stage
                    job = jobsWithIndices[i][4]
                    machineIndex = jobsWithIndices[i][3]  # rename this with "machineIndex"
                    availableMachines[0][machineIndex] = True
                    jFirst = removeFromJobs(jFirst, job)
                    minCurrentTime = getEndTimeOfJob(machineTracker, job)
                    if currentTime <= minCurrentTime:
                        currentTime = minCurrentTime
                # check if there exists finished job in stage 2.
                # if there exists finished job in stage 2, send it to processed jobs
                # remove job from jSecond
                # update availability of machine in second 2

                currentTime, jSecond = getAndProcessFinishedJobs(availableMachines, currentTime, jSecond, processedJobs,
                                                                 jFirst, qFirst)

                sendJobToSecondStage(avMacInSecondStage, availableMachines, currentTime, i, jSecond, job,
                                     machineTracker)

        # dealing with first stage
        sendJobsToFirstStage(availableMachines, currentTime, jFirst, machineTracker, qFirst)

        if len(jFirst) > 0:
            minEndTimeFromFirst = getMinimumEndTimeFromFirstStage(jFirst)
            if minEndTimeFromFirst >= currentTime:
                currentTime = minEndTimeFromFirst

        currentTime, jSecond = getAndProcessFinishedJobs(availableMachines, currentTime, jSecond, processedJobs, jFirst,
                                                         qFirst)

    return currentTime, machineTracker

def selection(pop):
    popObj = []
    for i in range(len(pop)):
        popObj.append([calculateObj(pop[i])[0], i])

    popObj.sort()

    probabilityDistribution = []
    probabilityDistributionIndex = []

    for i in range(len(pop)):
        probabilityDistributionIndex.append(popObj[i][1])
        prob = (2 * (i + 1)) / (len(pop) * (len(pop) + 1))
        probabilityDistribution.append(prob)

    parents = []
    for i in range(len(pop)):
        parents.append(list(np.random.choice(probabilityDistributionIndex, 2, p=probabilityDistribution)))

    return parents


def crossover(parents):
    # crossover points
    cp = list(np.random.permutation(np.arange(len(procTimes) - 1) + 1)[:2])

    # ordering crossover points in asc
    if cp[0] > cp[1]:
        t = cp[0]
        cp[0] = cp[1]
        cp[1] = t

    child = list(parents[0])

    # loop between crossover points
    for i in range(cp[0], cp[1]):
        child[i] = -1

    p = -1
    for i in range(cp[0], cp[1]):
        while True:
            p = p + 1
            if parents[1][p] not in child:
                child[i] = parents[1][p]
                break

    return child


def mutation(sol):
    # mutation points
    mp = list(np.random.permutation(np.arange(len(procTimes)))[:2])
    # 3, 6

    # sorting
    if mp[0] > mp[1]:
        t = mp[0]
        mp[0] = mp[1]
        mp[1] = t

    remJob = sol[mp[1]]

    for i in range(mp[1], mp[0], -1):
        sol[i] = sol[i - 1]

    sol[mp[0]] = remJob

    return sol


def elitistUpdate(oldPop, newPop):
    bestSolInd = 0
    bestSol = calculateObj(oldPop[0])[0]

    for i in range(1, len(oldPop)):
        tempObj = calculateObj(oldPop[i])[0]
        if tempObj < bestSol:
            bestSol = tempObj
            bestSolInd = i

    rndInd = random.randint(0, len(newPop) - 1)
    newPop[rndInd] = oldPop[bestSolInd]

    return newPop


def findBestSolution(pop):
    bestObj, machineTracker = calculateObj(pop[0])
    avgObj = bestObj
    bestInd = 0

    for i in range(1, len(pop)):
        tempObj = calculateObj(pop[i])[0]
        avgObj = avgObj + tempObj
        if tempObj < bestObj:
            bestObj = tempObj
            bestInd = i

    return bestInd, bestObj, avgObj / len(pop)


# Number of population
Npop = [2, 4, 6, 8, 10]
# Probability of crossover
Pc = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
# Probability of mutation
Pm = [0.01, 0.02, 0.03, 0.04, 0.05]
# Stopping number for generation
stopGeneration = [100, 200]


def genetic(pop, probCrossover, probMutation, stopGen):
    t1 = time.clock()
    population = initialization(pop)
    for i in range(stopGen):
        parents = selection(population)
        children = []

        for p in parents:
            r = random.random()
            if r < probCrossover:
                children.append(crossover([population[p[0]], population[p[1]]]))
            else:
                if r < 0.5:
                    children.append(population[p[0]])
                else:
                    children.append(population[p[1]])
        for c in children:
            r = random.random()
            if r < probMutation:
                mutation(c)

        population = elitistUpdate(population, children)

    bestSol, bestObj, avgObj = findBestSolution(population)
    t2 = time.clock()
    runningTime = t2 - t1

    return bestSol, bestObj, avgObj, runningTime


def startParameterTuning(pop, probCrossover, probMutation, stopGen):
    isFirst = True
    bestResult = {}
    results = []
    for n in pop:
        for pc in probCrossover:
            for pm in probMutation:
                for gen in stopGen:
                    params = {
                        "Npop": n,
                        "Pc": pc,
                        "Pm": pm,
                        "stopGen": gen
                    }
                    result = genetic(n, pc, pm, gen)
                    results.append({"result": result, "params": params})
                    print(result, params)
                    if isFirst:
                        bestResult = {
                            "result": result,
                            "params": params
                        }
                    else:
                        oldBestObj = bestResult['result'][1]
                        newBestObj = result[1]
                        if newBestObj <= oldBestObj:
                            bestResult = {
                                "result": result,
                                "params": params
                            }

                    isFirst = False

    return bestResult, results

startParameterTuning(Npop, Pc, Pm, stopGeneration)


C:\Users\obss\Anaconda3\lib\site-packages\ipykernel_launcher.py:384: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\Users\obss\Anaconda3\lib\site-packages\ipykernel_launcher.py:407: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


(0, 425, 425.0, 0.4164250999999979) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.01, 'stopGen': 100}
(0, 425, 425.0, 0.8823401000000004) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.01, 'stopGen': 200}
(0, 425, 425.0, 0.5082974999999976) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.02, 'stopGen': 100}
(0, 430, 430.0, 0.9607485999999952) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.02, 'stopGen': 200}
(0, 430, 430.0, 0.44175820000000243) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.03, 'stopGen': 100}
(0, 420, 420.0, 0.7985552999999967) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.03, 'stopGen': 200}
(0, 420, 420.0, 0.3663931999999974) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.04, 'stopGen': 100}
(0, 425, 425.0, 0.7704175999999947) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.04, 'stopGen': 200}
(0, 425, 425.0, 0.3847935000000007) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.05, 'stopGen': 100}
(0, 420, 420.0, 0.8676224000000019) {'Npop': 2, 'Pc': 0.5, 'Pm': 0.05, 'stopGen': 200}
(0, 420, 420.0, 0.3620197999999988) {'Npop': 2, 'Pc': 0.6, 'Pm': 0.01, 'stopGen': 100}
(0, 425, 425.0, 0.7817918999999947) {'Npop

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\obss\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-9c40b8e79760>", line 448, in <module>
    startParameterTuning(Npop, Pc, Pm, stopGeneration)
  File "<ipython-input-1-9c40b8e79760>", line 427, in startParameterTuning
    result = genetic(n, pc, pm, gen)
  File "<ipython-input-1-9c40b8e79760>", line 387, in genetic
    parents = selection(population)
  File "<ipython-input-1-9c40b8e79760>", line 275, in selection
    popObj.append([calculateObj(pop[i])[0], i])
  File "<ipython-input-1-9c40b8e79760>", line 223, in calculateObj
    availableMachines, currentTime, jFirst, jSecond, machineTracker, processedJobs, qFirst, qSecond = initiateCmax(sol)
  File "<ipython-input-1-9c40b8e79760>", line 143, in initiateCmax
    qFirst.put(sol[i])
  File "C:\Users\obss\Anaconda3\lib\queue.py", line 151, in put
    self.not_empty.not

KeyboardInterrupt: 